Final Project job response to request from Excel skills selected

In [1]:
# execute python getjobs script from excel
import sqlite3    
import os
import json
import win32com.client as win32
from pyxll import xl_func
@xl_func
# sktype: 'H'ard or 'S'oft skill
# dtype: 'I'D of skill or 'D'escription of skill

# getjobs function
# from excel =getjobs("python,communication,machine learning")
def getjobs(requestData): 
    import sqlite3     

# Select all SkillJobs rows
    sjsel_stmt = '''SELECT SkillsJobs_ID, SkillIDs, JobIDs FROM SkillsJobs;'''
    conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
    curs = conn.cursor()
    curs.execute(sjsel_stmt)
    sjs = curs.fetchall()
    conn.commit()

# convert request json to skill IDs list
    sks = {}
    selsk_stmt = '''SELECT Skill_ID, Description FROM Skills ORDER BY Skill_ID;'''
    for skkey, skval in curs.execute(selsk_stmt):
        sks[skkey] = skval
    skkey_list = list(sks.keys())
    skval_list = list(sks.values())   

# get hard and soft skills list
    hsklist = buildsklist('H','I')
    ssklist = buildsklist('S','I')
    hsktcnt = hsklist.count(',')+1
    ssktcnt = ssklist.count(',')+1

#  build user input skill list
    skids = ''
    for skill1 in str(requestData).split(","):
        position = skval_list.index(skill1.lower())
        skid = skkey_list[position]
        if skids == '':
            skids = str(skid)
        else:
            skids = skids + ',' + str(skid)

# search input skills in each SkillsJobs row, if found, 
# append title, company, location, skills required, % match 
# to response list
# process each SkillsJobs row
    jsel_stmt = '''SELECT JobTitle, Company, Location, URL FROM Jobs WHERE Job_ID = ?;'''
    outdicts = ''
    rows = []
    for sj in sjs:                          # loop through each SkillsJobs, get a skill job record
        sjIDs = sj[0]
        sjskIDs = sj[1]
        sjjobIDs = sj[2]
        thskcnt = 0
        tsskcnt = 0
        tskcnt = 0
        # get job total hard skill, soft skill and all skills count, by loop through each skill on skill job record
        for skid2 in sjskIDs.split(','):
            tskcnt += 1
            for hid2 in hsklist.split(','): 
                if skid2 == hid2: thskcnt += 1
            for sid2 in ssklist.split(','):
                if skid2 == sid2: tsskcnt += 1
        skdesc = ''
# process each input skill
        hskcnt = 0
        sskcnt = 0
        skcnt = 0  
        
        for skill2 in skids.split(','):          # loop through user skills        
            if skill2 in sjskIDs:                # if user skill in job skills            
                for hsk in hsklist.split(','):   # check if it's a hard skill            
                    if skill2 == hsk:
                        hskcnt += 1              # increment user hard skill match count
                for ssk in ssklist.split(','):   # check if it's a soft skill
                    if skill2 == ssk:
                        sskcnt += 1
        skcnt = hskcnt+sskcnt                    # total user skill count match with skills on skill job record
        if skcnt > 0:                            # if user skill matches in skills of the skill job record       
        # build job skills desc list
            for skill3 in sjskIDs.split(','):
                skill3 = int(skill3.strip())
                position = skkey_list.index(skill3)
                jsval = skval_list[position]
                if skdesc == '':
                    skdesc = jsval
                else:
                    skdesc = skdesc+ ','+jsval
            # get title, company, location, url     
            
            for jobID in sjjobIDs:      
                curs.execute(jsel_stmt, (jobID))
                jrec = curs.fetchone()
                if jrec is not None: 
                    title = jrec[0]
                    company = jrec[1]
                    location = jrec[2]
                    url = jrec[3]
                    hpercnt = 0; spercent = 0; opercent = 0
                    if thskcnt != 0:
                        hpercent = int(hskcnt / thskcnt * 100)
                    if tsskcnt != 0:   
                        spercent = int(sskcnt / tsskcnt * 100)
                    if tskcnt != 0:
                        opercent = int(skcnt / tskcnt * 100)
                    outdict = dict(Title=title, Company=company, Location=location, Skills=skdesc, URL=url, \
                            HPercent=hpercent, SPercent=spercent,OPercent=opercent)
                    outline = title+';'+company+';'+location+';'+str(hpercent)\
                            +';'+str(spercent)  +';'+str(opercent)+';'+skdesc 
                    row = list(outline.split(';'))
                    rows.append(row)

# launch Excel and display result in row, column format
    ExcelApp = win32.Dispatch('Excel.application')
    ExcelApp.Visible = True
    wb = ExcelApp.Workbooks.Add()
    ws = wb.Worksheets(1)
    header1 = requestData.replace('"','') 
    ws.Cells(1,1).Value = header1
    header2 = 'meeting job requirements from 160 jobs collected in Nov.-Dec. 2022'
    ws.Cells(2,1).Value = header2
    header_labels = ('Title','Company', 'Location','%Meet_HardskillReq','%Meet_SoftskillReq','%Meet_OverallskillReq','SkillsRequired')
# insert header labels
    for idx, val in enumerate(header_labels):
        ws.Cells(4,idx+1).Value = val

# insert data rows    
    row_tracker = 5
    column_size = len(header_labels)
    for row in rows:
        ws.Range(ws.Cells(row_tracker,1),ws.Cells(row_tracker,column_size)).value = row
        row_tracker += 1
# end of GetJobs(); housekepping 

def buildsklist(sktype,dtype):
        
    if sktype == 'H':
        selsk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('cs','ds','pltfm','stat','tech') ORDER BY DESCRIPTION;'''
    else:
        selsk_stmt = '''SELECT Skill_ID, Description FROM Skills WHERE TYPE IN ('ba','domn','nontech') ORDER BY DESCRIPTION;'''

# build skill dictionary {..."2":"Agile",...}
# technical  
    import sqlite3      
    conn = sqlite3.connect('C:\\DS_FinalProject_Data\\JobSkill.db')
    curs = conn.cursor()      
    curs.execute(selsk_stmt)
    skdescs = curs.fetchall()
    
    skmIDs = ''
    skills = ''
    for skid, skdesc in skdescs:
#         print(skid,' ',skdesc)
        if skmIDs == '':
            skmIDs = str(skid)
        else:
            skmIDs = skmIDs + ',' + str(skid)
            
        if skills == '':
            skills = skdesc
        else:
            skills = skills + ',' + skdesc
    if dtype == 'I':
        return skmIDs
    else:
        return skills 
    
#YY = getjobs("Python,Statistics,Communication")
# # print(YY)

